In [1]:
# !pip freeze > requirements.txt
# !pip install -r requirements.txt

In [2]:
import os
import json
from datetime import datetime
import pytz
from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from torchsummary import summary
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

import warnings
warnings.filterwarnings("ignore")


# Preprocessing

In [3]:
class PathLabelProcessor:
    def __init__(self, 
                 base_path,
                 folder_name,
                 pet_type,
                 lesion,
                 devices,
                 symptom):
        self.base_path = base_path
        self.folder_name = folder_name
        self.pet_type = pet_type
        self.lesion = lesion
        self.devices = devices
        self.symptom = symptom
        
        self.label_images()
        
    def find_folders_by_name(self):
        matching_folders = []

        for root, dirs, files in os.walk(self.base_path):
            for dir_name in dirs:
                if self.folder_name in dir_name:
                    folder_path = os.path.join(root, dir_name)
                    matching_folders.append(folder_path)

        for folder_path in matching_folders:
            print(folder_path)
            
        return matching_folders

    def find_image_json_pairs(self):
        image_paths = []
        json_paths = []

        for folder_path in self.find_folders_by_name():
            for root, dirs, files in os.walk(folder_path):
                for image_file in filter(lambda x: x.lower().endswith(('jpg', 'png')), files):
                    image_path = os.path.join(root, image_file)
                    json_file = f"{os.path.splitext(image_path)[0]}.json"
                    if os.path.isfile(json_file):
                        image_paths.append(image_path)
                        json_paths.append(json_file)

        print(f'Total images: {len(image_paths)}, Total JSON files: {len(json_paths)}')
        
        return image_paths, json_paths

    def label_images(self):
        self.labeled_image_paths = []

        for image_path, json_path in zip(*self.find_image_json_pairs()):
            with open(json_path) as f:
                data = json.load(f)
                
            is_symptomatic = data['label']['label_disease_lv_3'] in self.symptom and data['label']['label_disease_nm'] == self.lesion

            if data['images']['meta']['device'] in self.devices:
                if self.pet_type in os.path.dirname(image_path).lower():
                    self.labeled_image_paths.append((image_path, 0 if is_symptomatic else 1))
                else:
                    self.labeled_image_paths.append((image_path, 1))
        
        self.symptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 0))
        self.asymptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 1))

        print(f'Total cases: {len(self.labeled_image_paths)}')
        print(f'Number of symptomatic cases: {self.symptomatic_count}, Number of asymptomatic cases: {self.asymptomatic_count}')

class ImageDataset(Dataset):
    def __init__(self, labeled_image_paths, transform):
        self.labeled_image_paths = labeled_image_paths
        self.transform = transform

    def __len__(self):
        return len(self.labeled_image_paths)

    def __getitem__(self, idx):
        image_path, label = self.labeled_image_paths[idx]

        image = Image.open(image_path)
        image = self.transform(image)

        return image, label

class DataLoaderMaker:
    def __init__(self,
                 dataset,
                 batch_size,
                 train_ratio=0,
                 num_workers=None):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.num_workers = num_workers

        if train_ratio:
            self.split_and_make_dataloader()
        else:
            self.dataloader = self.make_dataloader(self.dataset)

    def make_dataloader(self, dataset, shuffle=False):
        dataloader = DataLoader(dataset=dataset,
                                batch_size=self.batch_size,
                                shuffle=shuffle,
                                num_workers=self.num_workers,
                                pin_memory=True)
        
        self.inspect_data(dataloader)
        
        return dataloader

    def split_and_make_dataloader(self):
        train_size = int(len(self.dataset) * self.train_ratio)
        test_size = len(self.dataset) - train_size
        train_dataset, test_dataset = random_split(self.dataset, [train_size, test_size])

        self.train_loader = self.make_dataloader(train_dataset, shuffle=True)
        self.test_loader = self.make_dataloader(test_dataset, shuffle=True)

    def inspect_data(self, dataloader):
        print("Inspecting Data...")

        class_counts = {}
        for _, labels in dataloader:
            labels = labels.type(torch.int)
            for label in labels.tolist():
                class_counts[label] = class_counts.get(label, 0) + 1

        print("- Class Counts:")
        for class_label, count in class_counts.items():
            print(f"  Class {class_label}: {count} samples")

In [4]:
%%time
base_path = 'eye/Train'
folder_name = '일반'
'''
['유']
dog: 안검염, 안검종양, 안검내반증, 유루증, 색소침착성각막염, 핵경화, 결막염
cat: 안검염, 결막염, 각막부골편, 비궤양성각막염, 각막궤양
['상', '하']
dog: 궤양성각막질환, 비궤양성각막질환
['초기', '비성숙', '성숙']
dog: 백내장
'''
pet_type = '개'
lesion = '안검종양'
devices = ['스마트폰', '일반카메라']
symptom = ['유']

processor = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                pet_type=pet_type,
                                lesion=lesion,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths
weight_class_0 = 1.0 / processor.symptomatic_count
weight_class_1 = 1.0 / processor.asymptomatic_count

eye/Train/고양이/안구/일반
eye/Train/개/안구/일반
Total images: 199816, Total JSON files: 199816
Total cases: 98646
Number of symptomatic cases: 52, Number of asymptomatic cases: 98594
CPU times: user 6.96 s, sys: 1.78 s, total: 8.75 s
Wall time: 8.76 s


In [5]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

CPU times: user 286 µs, sys: 104 µs, total: 390 µs
Wall time: 402 µs


In [6]:
%%time
batch_size = 96
num_workers = os.cpu_count()
train_ratio = 0.8

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              train_ratio=train_ratio)

Inspecting Data...
- Class Counts:
  Class (1,): 78880 samples
  Class (0,): 36 samples
Inspecting Data...
- Class Counts:
  Class (1,): 19714 samples
  Class (0,): 16 samples
CPU times: user 22.1 s, sys: 21.3 s, total: 43.4 s
Wall time: 53.6 s


# Modeling

In [7]:
class ModelTrainer:
    def __init__(self, 
                 model,
                 device,
                 train_dataloader,
                 valid_dataloader,
                 loss_fn,
                 optimizer,
                 scheduler):
        self.device = device
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.loss_fn = loss_fn.to(device)
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_f1_score = 0.0
        korea = pytz.timezone('Asia/Seoul')
        now = datetime.now(korea)
        start_time = now.strftime('%Y%m%d-%H%M%S')
        self.name = f'{start_time}_symptoms.pth'
        self.writer = SummaryWriter(log_dir=f'runs/{self.name}')

    def calculate_f1_score(self, predicted, labels):
        return f1_score(labels, predicted, average='binary')

    def calculate_auc_roc(self, predicted, labels):
        return roc_auc_score(labels, predicted)

    def train_one_epoch(self, epoch, num_epochs):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
    
        for step, (inputs, labels) in enumerate(tqdm(self.train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
    
            outputs = self.model(inputs)
            
            loss = self.loss_fn(outputs, labels)
            total_loss += loss.item()
    
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            loss.backward()
    
            self.optimizer.step()
            self.optimizer.zero_grad()
    
        self.scheduler.step()
        self.writer.add_scalar('LearningRate', self.scheduler.get_last_lr()[0], epoch)
    
        avg_loss = total_loss / len(self.train_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/train', avg_loss, epoch)        
        self.writer.add_scalar('Accuracy/train', accuracy, epoch)

    def eval_one_epoch(self, epoch):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in self.valid_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
        
                outputs = self.model(inputs)
        
                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()
        
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(self.valid_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/valid', avg_loss, epoch)
        self.writer.add_scalar('Accuracy/valid', accuracy, epoch)
        
        current_f1_score = self.calculate_f1_score(np.array(all_predicted), np.array(all_labels))
        current_auc_roc = self.calculate_auc_roc(np.array(all_predicted), np.array(all_labels))
        
        self.writer.add_scalar('F1 Score/valid', current_f1_score, epoch)
        self.writer.add_scalar('AUC-ROC/valid', current_auc_roc, epoch)
        
        if current_f1_score > self.best_f1_score:
            self.best_f1_score = current_f1_score
            torch.save(self.model, self.name)

    def train(self, num_epochs):
        for epoch in range(num_epochs):
            self.train_one_epoch(epoch, num_epochs)
            self.eval_one_epoch(epoch)
            
        self.writer.close()

In [8]:
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_ftrs = model.classifier[1].in_features
num_classes = 2
model.classifier[1] = nn.Linear(num_ftrs, num_classes)
class_weights = torch.tensor([weight_class_0, weight_class_1])
loss_fn = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

trainer = ModelTrainer(model=model,
                       device=device,
                       train_dataloader=data_loader.train_loader,
                       valid_dataloader=data_loader.test_loader,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       lesion=lesion)

In [9]:
trainer.train(30)

Epoch 1/30:   0%|          | 0/823 [00:00<?, ?batch/s]

Epoch 30/30: 100%|██████████| 823/823 [06:58<00:00,  1.97batch/s]


# Evaluation

In [15]:
class ModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = torch.load(path)
        self.model.to(self.device)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                logits = torch.nn.functional.sigmoid(outputs)
                predicted = (logits > 0.5).int()

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(logits.cpu().numpy())
                
        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Min Probability: {min_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")

In [5]:
%%time
base_path = 'eye/Valid'
folder_name = '일반'
'''
['유']
dog: 안검염, 안검종양, 안검내반증, 유루증, 색소침착성각막염, 핵경화, 결막염
cat: 안검염, 결막염, 각막부골편, 비궤양성각막염, 각막궤양
['상', '하']
dog: 궤양성각막질환, 비궤양성각막질환
['초기', '비성숙', '성숙']
dog: 백내장
'''
pet_type = '개'
lesion = '안검종양'
devices = ['스마트폰', '일반카메라']
symptom = ['유']

processor = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                pet_type=pet_type,
                                lesion=lesion,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

eye/Valid/고양이/안구/일반
eye/Valid/개/안구/일반
Total images: 24976, Total JSON files: 24976
Total cases: 13808
Number of symptomatic cases: 49, Number of asymptomatic cases: 13759
CPU times: user 938 ms, sys: 225 ms, total: 1.16 s
Wall time: 1.17 s


In [16]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

batch_size = 32
num_workers = os.cpu_count()

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = '20231206-151549_안검종양.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...


- Class Counts:
  Class (1,): 13759 samples
  Class (0,): 49 samples


  0%|          | 2/432 [00:01<03:40,  1.95it/s]

Batch Predictions: [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
Batch Labels: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Batch Probabilities: [[0.99940026]
 [0.99981505]
 [0.9994236 ]
 [0.99975616]
 [0.99969447]
 [0.9994388 ]
 [0.9997228 ]
 [0.9995042 ]
 [0.9994808 ]
 [0.999686  ]
 [0.9996679 ]
 [0.9993894 ]
 [0.99964523]
 [0.9996561 ]
 [0.999634  ]
 [0.9998919 ]
 [0.99947506]
 [0.9998124 ]
 [0.9996706 ]
 [0.99947494]
 [0.99949205]
 [0.99948394]
 [0.9996629 ]
 [0.9996685 ]
 [0.99961877]
 [0.99966204]
 [0.999658  ]
 [0.99957615]
 [0.99996686]
 [0.9997222 ]
 [0.9999311 ]
 [0.9997898 ]]
Batch Predictions: [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]

  1%|          | 5/432 [00:01<01:22,  5.17it/s]

...

 99%|█████████▉| 429/432 [00:41<00:00, 10.77it/s]

Batch Predictions: [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
Batch Labels: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Batch Probabilities: [[0.9999076 ]
 [0.99948573]
 [0.99968874]
 [0.9994898 ]
 [0.9996074 ]
 [0.9995921 ]
 [0.9996563 ]
 [0.9994234 ]
 [0.99960905]
 [0.9996475 ]
 [0.9999789 ]
 [0.9998499 ]
 [0.9997551 ]
 [0.99949265]
 [0.99952185]
 [0.99952817]
 [0.99950755]
 [0.99934703]
 [0.9997261 ]
 [0.9991379 ]
 [0.9996493 ]
 [0.9994831 ]
 [0.9997105 ]
 [0.99997497]
 [0.99914706]
 [0.9995254 ]
 [0.9995771 ]
 [0.99946994]
 [0.9995376 ]
 [0.99956733]
 [0.99958867]
 [0.9997131 ]]
Batch Predictions: [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]

100%|██████████| 432/432 [00:42<00:00, 10.26it/s]

Batch Predictions: [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
Batch Labels: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Batch Probabilities: [[0.99964774]
 [0.99963117]
 [0.99948657]
 [0.9995727 ]
 [0.9993832 ]
 [0.9997452 ]
 [0.99968755]
 [0.9994603 ]
 [0.9999027 ]
 [0.9995509 ]
 [0.99948514]
 [0.99934775]
 [0.99963176]
 [0.9994816 ]
 [0.99926275]
 [0.9995989 ]
 [0.99953806]
 [0.9996623 ]
 [0.9991295 ]
 [0.99952316]
 [0.99942696]
 [0.99969006]
 [0.99960893]
 [0.99957234]
 [0.9993299 ]
 [0.99996257]
 [0.9994444 ]
 [0.99966824]
 [0.9994875 ]
 [0.99942565]
 [0.9994967 ]
 [0.9996068 ]]
Batch Predictions: [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
Batch Labels: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]

Evaluation Results:
Confusion Matrix:
[[    0    49]
 [    0 13759]]
Accuracy: 0.9965
Precision: 0.9965
Recall: 1.0000
F1 Score: 0.9982
Min Probability: 0.9982
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0002
Mean Probability: 0.9996
CPU times: user 49.4 s, sys: 6.73 s, total: 56.2 s
Wall time: 53.5 s


In [7]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (0,): 49 samples


100%|██████████| 2/2 [00:00<00:00,  2.02it/s]

Evaluation Results:
Confusion Matrix:
[[49]]
Accuracy: 1.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 1.3 s, sys: 891 ms, total: 2.19 s
Wall time: 2.72 s


In [8]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...


- Class Counts:
  Class (1,): 13759 samples


100%|██████████| 430/430 [00:41<00:00, 10.34it/s]


Evaluation Results:
Confusion Matrix:
[[    0     0]
 [13759     0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 48.4 s, sys: 6.04 s, total: 54.4 s
Wall time: 52.2 s


In [ ]:
class PreModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.model = models.vgg16_bn(pretrained=True)
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = models.vgg16_bn(pretrained=True)
        nr_filters = self.model.classifier[0].in_features
        self.model.classifier = nn.Linear(nr_filters, 1)
        state_dict = torch.load(path, map_location=torch.device("cpu"))
        model_dict = self.model.state_dict()
        state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
        model_dict.update(state_dict)
        self.model.load_state_dict(model_dict)
        self.model = self.model.to(self.device)

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Min Probability: {min_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")

In [ ]:
%%time
dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = 'pre_eye.pt'

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)

In [ ]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

PreModelTester(path=path, device=device, dataloader=data_loader.dataloader)